In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import optuna
from optuna.samplers import TPESampler
import warnings
import shap
warnings.filterwarnings("ignore")

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [4]:
train = train.fillna('None').astype('string')
test = test.fillna('None').astype('string')

y = train['Depression']
X = train.drop(['Depression'], axis=1)

In [5]:
# iget this from optuna
catboost_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'learning_rate': 0.08114394459649094,
    'iterations': 1000,
    'depth': 6,
    'random_strength':0,
    'l2_leaf_reg': 0.7047064221215757,
    'random_seed':42,
    'verbose':False,
    'task_type': 'GPU'
}

In [6]:
cv = StratifiedKFold(5, shuffle=True, random_state=0)
cv_splits = cv.split(X, y)
scores = []
test_preds = []
X_test_pool = Pool(test, cat_features=X.columns.values)
for i, (train_idx, val_idx) in enumerate(cv_splits):
    model = CatBoostClassifier(**catboost_params)
    X_train_fold, X_val_fold = X.loc[train_idx], X.loc[val_idx]
    y_train_fold, y_val_fold = y.loc[train_idx], y.loc[val_idx]
    X_train_pool = Pool(X_train_fold, y_train_fold, cat_features=X.columns.values)
    X_valid_pool = Pool(X_val_fold, y_val_fold, cat_features=X.columns.values)
    model.fit(X=X_train_pool, eval_set=X_valid_pool, verbose=False, early_stopping_rounds=200)
    val_pred = model.predict(X_valid_pool)
    score = accuracy_score(y_val_fold, val_pred)
    scores.append(score)
    test_pred = model.predict_proba(X_test_pool)[:, 1]
    test_preds.append(test_pred)
    print(f'Fold {i + 1} accuracy_score: {score}')
print(f'Cross-validated accuracy_score: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')

Default metric period is 5 because AUC is/are not implemented for GPU


Fold 1 accuracy_score: 0.9414356787491116


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 2 accuracy_score: 0.9381307746979389


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 3 accuracy_score: 0.9416133617626155


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 4 accuracy_score: 0.9402985074626866


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 5 accuracy_score: 0.9391257995735608
Cross-validated accuracy_score: 0.940 +/- 0.001


In [7]:
print(f'Max accuracy_score score: {np.max(scores):.3f}')
print(f'Min accuracy_score score: {np.min(scores):.3f}')

Max accuracy_score score: 0.942
Min accuracy_score score: 0.938


In [8]:
test.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,140700,Shivam,Male,53.0,Visakhapatnam,Working Professional,Judge,None,2.0,None,None,5.0,Less than 5 hours,Moderate,LLB,No,9.0,3.0,Yes
1,140701,Sanya,Female,58.0,Kolkata,Working Professional,Educational Consultant,None,2.0,None,None,4.0,Less than 5 hours,Moderate,B.Ed,No,6.0,4.0,No
2,140702,Yash,Male,53.0,Jaipur,Working Professional,Teacher,None,4.0,None,None,1.0,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No
3,140703,Nalini,Female,23.0,Rajkot,Student,None,5.0,None,6.84,1.0,None,More than 8 hours,Moderate,BSc,Yes,10.0,4.0,No
4,140704,Shaurya,Male,47.0,Kalyan,Working Professional,Teacher,None,5.0,None,None,5.0,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No


In [10]:
np.round(np.mean(test_preds, axis=0))

array([0., 0., 0., ..., 0., 1., 0.])

In [12]:
submission = pd.DataFrame(data={
    "id": np.arange(140700, 140700 + len(test)),
    "Depression": np.round(np.mean(test_preds, axis=0))
})

submission.to_csv("submission/catboost.csv", index=False)